일반화 성능도 있기 때문에 전체 데이터셋에 대해서 성능 평가를 해봐야 함

In [1]:
import pickle
with open('../models/SVC (C=0.1) norml30_r15.pkl', 'rb') as f:
    classifier = pickle.load(f)
    
from py.utils import load_data
head = 'l30_r15'
directory = '../data/'
x, y, x_words, vocabs = load_data(head, directory)
x = x.tocsr()

x shape = (15166, 2617)
y shape = (15166,)
# features = 2617
# L words = 15166


In [2]:
def get_sample_r(li, topk=5):
    nonzero = x[li,:].nonzero()[1]
    base = min(50, len(nonzero)//2)
    return [vocabs[r] for r in sorted(nonzero, key=lambda x:word_frequency[0,x], reverse=True)[base:base+topk]]

word_frequency = x.sum(axis=0)
word_frequency

matrix([[ 52,  18,  16, ...,  22,  75, 234]], dtype=int64)

In [3]:
# sparse matrix type
print(classifier.dual_coef_.shape)

# sum = 0 ?? alpha sum은 1아닌가?
print(classifier.dual_coef_.data.sum())

(1, 5012)
0.0


In [4]:
# Support vectors
classifier.support_vectors_.shape

(5012, 2617)

In [9]:
# Proportion of Support vector 
classifier.n_support_.sum() / x.shape[0]

0.33047606488197284

In [10]:
# Vector row id
classifier.support_

array([    5,     9,    13, ..., 15138, 15150, 15151], dtype=int32)

In [11]:
base = 0
topk = 40
for idx, l in sorted(enumerate(classifier.support_), key=lambda x:abs(classifier.dual_coef_[0,x[0]]), reverse=True)[base :base + topk]:
    print('%s%s - %s (%.3f)' % (x_words[l], '' if y[l] == 1 else '(다)', get_sample_r(l), classifier.dual_coef_[0,idx]))

피나(다) - ['게'] (-0.100)
되묻(다) - ['는데', '기도', '자', '더니'] (-0.100)
버티(다) - ['기는', '면서도', '듯이', '었기', '었지만'] (-0.100)
빠져나오(다) - ['지도', '려고', '려는', '듯', '기는'] (-0.100)
귀찮(다) - ['단', '으니까', '겠지만', '은데', '으면서도'] (-0.100)
미치(다) - ['곤', '기가', '겠다고', '긴', '겠지만'] (-0.100)
가진(다) - ['만큼', '다거나', '다든지', '데다', '대요'] (-0.100)
사로잡(다) - ['았던', '으면서', '기에', '는다는', '겠다는'] (-0.100)
버려(다) - ['지기', '두고', '지게', '졌던', '대는'] (-0.100)
부릅뜨(다) - ['니', '고는', '니까', '신', '고서'] (-0.100)
두드러지(다) - ['기', '던', '며', '면서', '지만'] (-0.100)
드리(다) - ['겠다는', '듯이', '곤', '기가', '기만'] (-0.100)
폭넓(다) - ['지', '지는', '고도', '음', '음에'] (-0.100)
어지럽(다) - ['도록', '기도', '지도', '더니', '기만'] (-0.100)
내보이(다) - ['기도', '지는', '니까', '기만', '려'] (-0.100)
미쳐(다) - ['가고', '보는', '버릴', '버리고', '보지'] (-0.100)
하찮(다) - ['게', '아', '기조차'] (-0.100)
늘어났(다) - ['는데', '으며', '는지', '으나', '음을'] (-0.100)
저러(다) - ['진', '지도', '는지', '면서도', '든지'] (-0.100)
애써(다) - ['온', '야만', '보지', '왔지만', '줘서'] (-0.100)
끌어당기(다) - ['자', '듯', '면서도', '고서', '느라'] (-0.100)
꺼(다) - ['야지', '보는', '버리고'

In [12]:
from collections import Counter
alpha_count = Counter(classifier.dual_coef_.data)
print('number of alphas = %d' % len(alpha_count))
print('number of support vectors = %d\n' % len(classifier.dual_coef_.data))

for alpha, count in sorted(alpha_count.items(), key=lambda x:(x[1], abs(x[0])), reverse=True)[:50]:
    print('alpha= %.3f, count=%d (%.3f)' % (alpha, count, count/classifier.n_support_.sum()))

number of alphas = 2
number of support vectors = 5012

alpha= -0.100, count=2506 (0.500)
alpha= 0.100, count=2506 (0.500)


In [13]:
classifier.n_support_

array([2506, 2506], dtype=int32)

Support Vector Machine은 약 30%의 매우 높은 support vector를 모델에 저장하고 있으며, 각각 support vector의 가중치 역시 다르지 않다. negative는 모두 -0.1, positive는 모두 0.1 weight를 가지고 있다. k-NN classifier와 다르지 않을 정도의 support vector ratio이며, 질 좋은 support vector를 선택하지도 (equal alpha) 않는다. 

## Regularization을 약하게 하는 경우

In [14]:
with open('../models/Support Vector Machine (rbf, C=10.0) norml30_r15.pkl', 'rb') as f:
    classifier = pickle.load(f)
    
from py.utils import load_data
head = 'l30_r15'
directory = '../data/'
x, y, x_words, vocabs = load_data(head, directory)
x = x.tocsr()

x shape = (15166, 2617)
y shape = (15166,)
# features = 2617
# L words = 15166


In [15]:
# Proportion of Support vector 
classifier.n_support_.sum() / x.shape[0]

0.20440458921271265

In [16]:
from collections import Counter
alpha_count = Counter(classifier.dual_coef_.data)
print('number of alphas = %d' % len(alpha_count))
print('number of support vectors = %d\n' % len(classifier.dual_coef_.data))

for alpha, count in sorted(alpha_count.items(), key=lambda x:(x[1], abs(x[0])), reverse=True)[:50]:
    print('alpha= %.3f, count=%d (%.3f)' % (alpha, count, count/classifier.n_support_.sum()))

number of alphas = 22
number of support vectors = 3100

alpha= -10.000, count=1542 (0.497)
alpha= 10.000, count=1538 (0.496)
alpha= 9.566, count=1 (0.000)
alpha= 7.743, count=1 (0.000)
alpha= 7.300, count=1 (0.000)
alpha= 7.253, count=1 (0.000)
alpha= 6.767, count=1 (0.000)
alpha= -5.719, count=1 (0.000)
alpha= -5.715, count=1 (0.000)
alpha= 4.977, count=1 (0.000)
alpha= 4.967, count=1 (0.000)
alpha= 4.852, count=1 (0.000)
alpha= -4.852, count=1 (0.000)
alpha= -4.775, count=1 (0.000)
alpha= 4.195, count=1 (0.000)
alpha= -3.559, count=1 (0.000)
alpha= 2.747, count=1 (0.000)
alpha= 2.700, count=1 (0.000)
alpha= 2.577, count=1 (0.000)
alpha= 2.156, count=1 (0.000)
alpha= -2.129, count=1 (0.000)
alpha= -1.052, count=1 (0.000)


Regularization 을 약하게 걸어도 대부분은 -10, 10의 weight를 가지며, support vector ratio가 20%로 줄어들었지만 여전히 많은 숫자이다. instance learning과 다르지 않다